In [ ]:
import requests
from tqdm.auto import tqdm
import json

In [ ]:
sber = '3529'
page = 1
num_per_page = 100
moscow = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&per_page={num_per_page}'

In [ ]:
res = requests.get(url)
vacancies = res.json()
num_pages = vacancies.get('pages')
num_pages

In [ ]:
all_vacancy_ids= []
for i in tqdm(range(vacancies.get('pages'))):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)

In [ ]:
# Задание вытащить все вакансии СБЕРа с ХХ (у апи есть ограничение в 2000, подумайте как его обойти)
# Вытащите все полные описания этих вакансий по запросу с https://api.hh.ru/vacancies/43326020 где 43326020 id вакансии
# Создайте аналогичный vacancy DataFrame только добавьте поле skills
# Переведите даты публикаций в datetime
# Постройте график опубликованных вакансий по датам
# Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий
# Найдите те вакансии с использованием python, которые вам интересны
# Определите по полю skills какие навыки больше всего востребованы для этих вакансий, и
# Постройте график наиболее востребованных вакансий

In [ ]:
res = requests.get('https://api.hh.ru/vacancies/54162408')
res.json()

Вытащите все полные описания этих вакансий

In [ ]:
describe = dict()


for i, j in enumerate(all_vacancy_ids):
    if i < 10:
        res = requests.get(f'https://api.hh.ru/vacancies/{j}')
        describe[j] = res.json().get('description')

Создайте аналогичный vacancy DataFrame только добавьте поле skills

In [ ]:
import pandas as pd

des = dict()

for i, j in enumerate(all_vacancy_ids):
    if i < 10:
        res = requests.get(f'https://api.hh.ru/vacancies/{j}')
        des[j] = res.json().get('key_skills')

   
    
df = pd.DataFrame(describe.values(), describe.keys(), columns=['description']).reset_index()
df1 = pd.DataFrame(des.values(), des.keys(), columns=['key_skills']).reset_index()
df = df.merge(df1, on='index', how="left")
df.head(5)
  

Переведите даты публикаций в datetime
Постройте график опубликованных вакансий по датам

In [ ]:
dtc = dict()

for i, j in enumerate(all_vacancy_ids):
    if i < 10:
        res = requests.get(f'https://api.hh.ru/vacancies/{j}')
        dtc[j] = res.json().get('published_at')


In [ ]:
import datetime as dt

dr = pd.DataFrame(dtc.keys(), dtc.values(),  columns=['id'])

dr.index = pd.to_datetime(dr.index)
dr.loc[:, 'index'] = df.loc[:, 'index'].dt.year
dr.head(5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn') 

dd = dr.groupby(dr.index)['id'].count().reset_index()
sns.lineplot(dd.index, dd.id)
plt.show()

Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий

In [ ]:
dk = dr.copy()
dk = dk.index.strftime('%A')
dk.groupby('index')['id'].count().sort_values(ascending=False).reset_index()

Найдите те вакансии с использованием python, которые вам интересны

In [ ]:
fave = dict()
for i, j in enumerate(all_vacancy_ids):
    if i < 10:
        res = requests.get(f'https://api.hh.ru/vacancies/{j}')
        if 'python' in res.json().get('key_skills').lower and 'data' in res.json().get('name').lower:
            fave[j] = res.json().get('name')
fave.keys()